<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.1/533.1 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=7fd8551d20e2c20b918f07b98152b6cf4371afff7dc78c6b3722106f575e17e4
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-11 20:09:29
-------------------
qualified stocks: 84
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  5.59 L
Current:  1.43 C
-------------------
Today PnL: 26.08 K (0.18%)
Current PnL: -22.22 L (-14.56%)
CY Booked + Current PnL: -8.73 L (-5.72%)
-------------------
Total profit:  1.38 L
Total loss:  -23.60 L
-------------------
Total Booked + Current PnL: 18.25 L (14.53%)
Total Booked PnL: 40.47 L (32.22%)
Curr Year Booked PnL: 13.49 L (9.42%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.30 C
Est FTT PnL: 86.88 L (60.67%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,101.28,51.0,M-SC,2.35,86676.0,-14101.0,14198.0,0.15,-13.99,16.38,0.09,245.0,-0.99,0.63,13.87,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.15,88650.0,-2028.0,16932.0,0.15,-2.24,19.10,16.44,101.0,-0.12,0.64,17.49,X40,ATH,PHARMA
41,ITC,452.00,-38.34,46.0,X-LC,0.83,198950.0,-1188.0,22083.0,0.32,-0.59,11.10,10.44,4.0,-0.05,1.45,4.94,X40,NTT,FMCG
33,ICICIGI,2252.93,-14.07,54.0,X-MC,6.59,185637.0,11317.0,23891.0,-1.45,6.49,12.87,20.19,91.0,0.47,1.35,22.59,X40,ATH,INSURANCE
37,INDIGOPNTS,1408.00,127.68,78.0,M-SC,5.23,149116.0,-25443.0,25469.0,4.09,-14.58,17.08,0.02,221.0,-1.00,1.08,29.71,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3591.89,53.0,M-SC,3.81,165818.0,-31314.0,121147.0,9.41,-15.88,73.06,45.57,236.0,-0.26,1.21,17.56,XY24,NTT,AUTO
37,INDIGOPNTS,1408.00,127.68,78.0,M-SC,5.23,149116.0,-25443.0,25469.0,4.09,-14.58,17.08,0.02,221.0,-1.00,1.08,29.71,OX40N,NTT,PAINTS
38,INDUSINDBK,1800.00,283.75,71.0,L-MC,7.37,47088.0,-30118.0,55512.0,3.35,-39.01,117.89,32.89,258.0,-0.54,0.34,29.68,XR,NTT,BANKS
14,BSOFT,836.99,-13.25,61.0,H-SC,9.72,97576.0,-43077.0,115862.0,2.19,-30.63,118.74,51.74,131.0,-0.37,0.71,12.89,XR,ATH,IT
13,BERGEPAINT,680.00,-17.69,56.0,X-MC,1.09,221211.0,-6128.0,54197.0,1.98,-2.70,24.50,21.14,106.0,-0.11,1.61,24.69,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,WHIRLPOOL,2270.00,-45.08,37.0,M-SC,8.35,113887.0,-911.0,94959.0,-3.68,-0.79,83.38,81.92,223.0,-0.01,0.83,34.82,XR,NTT,DURABLES
26,GREENPANEL,537.00,195.87,36.0,M-SC,5.21,132224.0,-50854.0,129831.0,-2.92,-27.78,98.19,43.14,230.0,-0.39,0.96,24.37,XY24,NTT,MISC
52,PGHH,17907.65,-33.06,30.0,X-MC,0.71,193770.0,-7050.0,74853.0,-2.76,-3.51,38.63,33.76,80.0,-0.09,1.41,1.50,X40,ATH,FMCG
81,VOLTAS,1530.00,-6.81,36.0,X-MC,4.02,195570.0,3828.0,33931.0,-2.75,2.00,17.35,19.69,99.0,0.11,1.42,8.93,XY25,NTT,AC
53,QUESS,986.00,-52.07,31.0,X-SC,37.28,47849.0,-17157.0,168084.0,-2.42,-26.39,351.28,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,WHIRLPOOL,2270.0,-45.08,37.0,M-SC,8.35,113887.0,-911.0,94959.0,-3.68,-0.79,83.38,81.92,223.0,-0.01,0.83,34.82,XR,NTT,DURABLES
83,WIPRO,420.0,-14.70,49.0,M-LC,5.67,149848.0,-1097.0,110558.0,0.77,-0.73,73.78,72.51,53.0,-0.01,1.09,5.25,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,101.28,51.0,M-SC,2.35,86676.0,-14101.0,14198.0,0.15,-13.99,16.38,0.09,245.0,-0.99,0.63,13.87,OX40N,NTT,DURABLES
37,INDIGOPNTS,1408.0,127.68,78.0,M-SC,5.23,149116.0,-25443.0,25469.0,4.09,-14.58,17.08,0.02,221.0,-1.00,1.08,29.71,OX40N,NTT,PAINTS
45,KANSAINER,340.0,-68.16,40.0,H-SC,2.27,219465.0,-50202.0,86535.0,-0.80,-18.62,39.43,13.47,138.0,-0.58,1.59,11.59,XY24,NTT,PAINTS
65,SIS,528.0,2010.36,45.0,H-SC,2.23,85170.0,-25862.0,49467.0,-0.22,-23.29,58.08,21.26,156.0,-0.52,0.62,15.07,OX40N,NTT,MISC
42,JCHAC,2282.0,19943.48,63.0,M-SC,1.49,97955.0,-29850.0,29837.0,0.05,-23.36,30.46,-0.01,232.0,-1.00,0.71,10.92,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.08,129956.0,6620.0,125005.0,0.01,5.37,96.19,106.72,119.0,0.05,0.94,30.00,AR,ATH,MISC
60,SAMMAANCAP,326.00,-192.05,53.0,M-SC,35.14,158724.0,9504.0,134677.0,-1.61,6.37,84.85,96.62,208.0,0.07,1.15,71.77,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.08,129956.0,6620.0,125005.0,0.01,5.37,96.19,106.72,119.0,0.05,0.94,30.00,AR,ATH,MISC
83,WIPRO,420.00,-14.70,49.0,M-LC,5.67,149848.0,-1097.0,110558.0,0.77,-0.73,73.78,72.51,53.0,-0.01,1.09,5.25,XR,NTT,IT
82,WHIRLPOOL,2270.00,-45.08,37.0,M-SC,8.35,113887.0,-911.0,94959.0,-3.68,-0.79,83.38,81.92,223.0,-0.01,0.83,34.82,XR,NTT,DURABLES
46,KPIGREEN,731.05,2.88,44.0,H-SC,9.78,122548.0,-2749.0,61678.0,0.41,-2.19,50.33,47.03,141.0,-0.04,0.89,53.44,MH,ATH,POWER
35,IEX,219.00,-33.86,46.0,H-SC,15.97,189084.0,-10870.0,108099.0,-0.17,-5.44,57.17,48.63,115.0,-0.10,1.37,5.49,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-19.16,16.0,X-MC,5.55,248535.0,-5361.0,87658.0,-2.26,-2.11,35.27,32.42,105.0,-0.06,1.81,12.67,X40,ATH,FMCG
72,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.08,5.01,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.50,86893.0,-41777.0,87075.0,-0.25,-32.47,100.21,35.20,219.0,-0.48,0.63,0.00,X40,NTT,FOOTWEAR
52,PGHH,17907.65,-33.06,30.0,X-MC,0.71,193770.0,-7050.0,74853.0,-2.76,-3.51,38.63,33.76,80.0,-0.09,1.41,1.50,X40,ATH,FMCG
53,QUESS,986.00,-52.07,31.0,X-SC,37.28,47849.0,-17157.0,168084.0,-2.42,-26.39,351.28,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-33.06,30.0,X-MC,0.71,193770.0,-7050.0,74853.0,-2.76,-3.51,38.63,33.76,80.0,-0.09,1.41,1.50,X40,ATH,FMCG
41,ITC,452.00,-38.34,46.0,X-LC,0.83,198950.0,-1188.0,22083.0,0.32,-0.59,11.10,10.44,4.0,-0.05,1.45,4.94,X40,NTT,FMCG
64,SIEMENS,4671.50,-6.04,37.0,H-LC,1.62,151100.0,-34995.0,82470.0,-1.06,-18.80,54.58,25.51,15.0,-0.42,1.10,11.35,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-9.16,36.0,X-MC,3.31,303156.0,-24835.0,131357.0,-0.97,-7.57,43.33,32.48,92.0,-0.19,2.20,2.99,X40,ATH,ELECTRICAL
77,UNITDSPR,1644.00,-7.24,59.0,X-LC,3.62,243294.0,9150.0,39462.0,0.35,3.91,16.22,20.77,86.0,0.23,1.77,11.10,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.50,86893.0,-41777.0,87075.0,-0.25,-32.47,100.21,35.20,219.0,-0.48,0.63,0.00,X40,NTT,FOOTWEAR
53,QUESS,986.00,-52.07,31.0,X-SC,37.28,47849.0,-17157.0,168084.0,-2.42,-26.39,351.28,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
52,PGHH,17907.65,-33.06,30.0,X-MC,0.71,193770.0,-7050.0,74853.0,-2.76,-3.51,38.63,33.76,80.0,-0.09,1.41,1.50,X40,ATH,FMCG
1,ABB,7934.00,-42.78,32.0,H-LC,4.98,238968.0,-22651.0,141875.0,-0.27,-8.66,59.37,45.57,7.0,-0.16,1.74,1.68,AR,NTT,ELECTRICAL
10,BAJAJHFL,181.50,-16.17,36.0,X-MC,6.36,176595.0,-23911.0,121056.0,-0.54,-11.93,68.55,48.45,90.0,-0.20,1.28,1.83,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-52.07,31.0,X-SC,37.28,47849.0,-17157.0,168084.0,-2.42,-26.39,351.28,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
57,RELAXO,1176.00,-40.46,43.0,X-SC,4.02,81197.0,-63963.0,139894.0,0.92,-44.06,172.29,52.31,136.0,-0.46,0.59,8.69,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.50,86893.0,-41777.0,87075.0,-0.25,-32.47,100.21,35.20,219.0,-0.48,0.63,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.25,47.0,X-MC,3.15,88650.0,-2028.0,16932.0,0.15,-2.24,19.10,16.44,101.0,-0.12,0.64,17.49,X40,ATH,PHARMA
32,HONAUT,58357.33,-24.48,36.0,X-SC,0.92,106515.0,-21423.0,68553.0,-1.42,-16.75,64.36,36.84,143.0,-0.31,0.77,9.05,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-28.04,54.0,X-LC,9.35,286418.0,-59538.0,126253.0,0.72,-17.21,44.08,19.28,1.0,-0.47,2.08,5.88,X40,ATH,IT
39,INFY,2275.00,-16.37,64.0,X-LC,4.70,325954.0,13066.0,158609.0,1.11,4.18,48.66,54.87,3.0,0.08,2.37,12.80,X40,BTT,IT
41,ITC,452.00,-38.34,46.0,X-LC,0.83,198950.0,-1188.0,22083.0,0.32,-0.59,11.10,10.44,4.0,-0.05,1.45,4.94,X40,NTT,FMCG
80,VBL,671.64,-16.58,53.0,X-LC,5.73,300243.0,-15599.0,128264.0,1.59,-4.94,42.72,35.67,5.0,-0.12,2.18,8.23,X40N,ATH,FMCG
1,ABB,7934.00,-42.78,32.0,H-LC,4.98,238968.0,-22651.0,141875.0,-0.27,-8.66,59.37,45.57,7.0,-0.16,1.74,1.68,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,27.23,76608.0,-36941.0,76991.0,-1.82,-32.53,100.50,35.27,268.0,-0.48,0.56,87.50,XR,NTT,HOTELS
7,ASIANTILES,137.00,7555.56,67.0,L-SC,14.06,83970.0,-9840.0,86456.0,-0.13,-10.49,102.96,81.67,269.0,-0.11,0.61,62.61,XR,NTT,CERAMICS
49,MASFIN,398.61,-20.05,46.0,H-SC,7.91,90735.0,-7245.0,28845.0,-0.84,-7.39,31.79,22.05,152.0,-0.25,0.66,31.89,XR,ATH,FINANCE
46,KPIGREEN,731.05,2.88,44.0,H-SC,9.78,122548.0,-2749.0,61678.0,0.41,-2.19,50.33,47.03,141.0,-0.04,0.89,53.44,MH,ATH,POWER
36,INDIAMART,4810.62,-53.52,53.0,H-SC,5.08,129956.0,6620.0,125005.0,0.01,5.37,96.19,106.72,119.0,0.05,0.94,30.00,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7555.56,67.0,L-SC,14.06,83970.0,-9840.0,86456.0,-0.13,-10.49,102.96,81.67,269.0,-0.11,0.61,62.61,XR,NTT,CERAMICS
78,VAIBHAVGBL,521.00,67.49,57.0,H-SC,5.62,145258.0,-37517.0,148584.0,-1.77,-20.53,102.29,60.77,125.0,-0.25,1.06,30.82,XR,NTT,JEWELLERY
37,INDIGOPNTS,1408.00,127.68,78.0,M-SC,5.23,149116.0,-25443.0,25469.0,4.09,-14.58,17.08,0.02,221.0,-1.00,1.08,29.71,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-7.31,74.0,X-LC,6.18,233834.0,-17934.0,71156.0,0.26,-7.12,30.43,21.13,27.0,-0.25,1.70,25.78,X40,ATH,PAINTS
5,ANGELONE,3033.00,22.45,68.0,X-SC,6.67,216595.0,25589.0,32121.0,1.34,13.40,14.83,30.21,157.0,0.80,1.57,37.08,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.54
1,25,46.00
2,50,77.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,44.98
LC,30.01
MC,25.02


In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,28.03
X40,19.93
X40N,12.87
XY25,10.56
XR,10.02
AR,7.97
OX40N,6.83
X200,1.87
SR,1.04


In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
H-SC,24.69
X-MC,22.27
X-LC,21.73
M-SC,13.02
X-SC,5.75
H-LC,4.71
M-LC,2.54
H-MC,2.41
L-SC,1.52


In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.57,-4.15,37.83
IT,13.08,-16.80,79.33
FINANCE,12.53,-9.71,59.44
MISC,7.94,-18.87,78.83
PAINTS,5.98,-12.11,28.82
ELECTRICAL,5.81,-12.99,53.05
INSURANCE,4.62,0.48,34.59
AUTO,2.91,-43.51,103.41
BANKS,2.80,-38.76,129.19


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3141126.0,20
XR,1278847.0,13
AR,1190013.0,9
X40,949767.0,13
X40N,695832.0,9
OX40N,545291.0,9
XY25,481941.0,7
SR,276820.0,2
X200,67121.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3433358.0,24
X-MC,1344462.0,15
M-SC,1253346.0,15
X-LC,1050211.0,12
X-SC,556679.0,6
H-LC,291466.0,3
H-MC,255687.0,2
L-SC,253233.0,3
M-LC,156736.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1188470.0      6
           AR         841581.0      5
           XR         805290.0      7
M-SC       XY24       728229.0      6
X-MC       XY24       566116.0      4
X-LC       X40        433364.0      5
X-MC       X40        429328.0      7
X-LC       XY24       308213.0      2
X-SC       X40N       301520.0      4
H-SC       SR         276820.0      2
           OX40N      259519.0      3
X-LC       X40N       233696.0      3
H-LC       AR         224345.0      2
X-MC       XY25       188402.0      2
H-MC       XY24       182014.0      1
X-SC       XY24       168084.0      1
L-SC       XR         163447.0      2
X-MC       X40N       160616.0      2
M-SC       XR         144040.0      2
           XY25       134677.0      1
           AR         124087.0      2
           OX40N      122313.0      4
M-LC       XR         110558.0      1
L-SC       OX40N       89786.0      1
X-SC       X40         87075.0      1
X-LC       XY25        74938.0      2
H-MC       OX40N       73673.0      1
H-LC       X200        67121.0      1
H-SC       MH          61678.0      1
L-MC       XR          55512.0      1
M-LC       XY25        46178.0      1
L-LC       XY25        37746.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
